# Question 1

The file contains 139 000 000 strings with 32 hexadecimal characters.

In [1]:
from tqdm import tqdm
file=open("hash.txt","r")
line=file.readline()
file.close()

In [2]:
line

'844082e02a27ddee8d99ea1af94a2969\n'

Observe that it also read the "\n" character. We will not consider it during the hasing later.

## Hashing function

We use a simple hashing function for this problem since all the strings are quite long. This is also fast to compute.
Given a string $s=c_1c_2...c_{32}$, we define $my\_hash(s)=h+\sum_{n=1}^{32} ord(c_i)\cdot p^{i-1} \ \ \ mod\ 2^{64}$, where $ord()$ is the ASCII value given by the built-in python function while $p$ and $h$ are two fixed primes. We chose $h=7$ and $p=31$ for our purposes, but higher primes can be used. 

In [3]:
p=31
const=2**64
#Since all the strings we're hashing have 32 characters, we can precompute some powers of the prime.
#For 32 digits numbers it is only necessary to use the first 32 powers (including the 0th power).
powers=[1]
for i in range(1,64):
    powers.append((powers[i-1]*p)%const)


In [4]:
def my_hash(string):
    h = 7;
    for i in range(len(string)):
        h +=  ord(string[i])*powers[i]
        #h = h%const
    return h%const

In [5]:
my_hash("844082e02a27ddef8d99ea1af94a2969")

10509802827583165011

The output has to be converted to binary for the HyperLogLog algorithm

## HyperLogLog

HyperLogLog is an algorithm that uses hashing to approximate the number of distinct elements in a list. The main advantage is that it achieves a very high accuracy while using very little memory compared to the naive approach of putting all the elements in a set. For more information, see [here](http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf).

This function adds leading zeros if the binary number doesn't have any.

In [6]:
def add_zeros(string):
    return "0"*(64-len(string))+string

We initialize the registers for the HLL data structure. 

In [7]:
m=16 #This is the number of bits used for buckets 

#Let's define a vector of 2**m integers for the HLL:
registers=[0 for x in range(2**m)]

This function takes as input a binary number in string format (because we want to keep the leading zeros) and returns the index of the corresponding register.

In [8]:
def bucket_index(string):
    temp=string[:m]
    temp=int(temp, 2)
    return temp

This function counts the number of zeros after the digits used to select the register. 

In [9]:
def num_leading_zeros(string):
    string=string[m:]
    maximum=0
    temp=0
    for i in range(len(string)):
        if string[i]=="0":
            temp+=1
        else:
            maximum=temp
            break
            
    return maximum
        
        

We can finally use all these functions to implement the HLL on the "hash.txt" file.

In [10]:
file=open("F:\hash.txt","r")
for line in tqdm(file):
    hash_value=add_zeros(("{0:b}".format(my_hash(line[:-1]))))
    registers[bucket_index(hash_value)]=max(registers[bucket_index(hash_value)],num_leading_zeros(hash_value))
file.close()

139000000it [21:22, 108358.95it/s]


In [11]:
a_m = 0.7213/(1+1.079/(2**m))
#This is a good approximation for the $a_m$ value in the HyperLogLog Algorithm

We finally compute the expected value.

In [12]:
sum=0
for i in range(len(registers)):
    sum+=2**(-registers[i]-1) 
    #The "-1" is here because we count the number of consecutive zeroes, while the HLL algorithm asks
    #for the first "1" appearing, which is the next position
sum=sum**(-1)
expected_value=(2**m)**2*a_m*sum

In [13]:
expected_value

124263775.63713567

## Exact calculation

We use a set to avoid counting duplicates more than once. Be careful if trying to execute this: this method is somewhat memory intensive.

In [14]:
file=open("F:\hash.txt","r")
temp=set()
for line in tqdm(file):
    temp.add(line)
    
file.close()

139000000it [01:55, 1204816.02it/s]


In [15]:
len(temp)

125000000

In [35]:
#We finally compute the error:
print("The absolute error is", abs(int(expected_value-len(temp))))
print("The relative error is", round(abs((expected_value-len(temp))/len(temp)),3)*100,"%")

The absolute error is 736224
The relative error is 0.6 %


# Question 3

Let $A$ be the array we want to sort.

In [17]:
A=[1,2,4,2,1,3,5,7,1,2,6,9,11,2,4,5]

Calculating the maximum and the minimum of the array can be done with a single pass. The complexity for this step is $O(n)$ where $n$ is the lenght of the array.

In [18]:
m=A[0]
M=A[0]
for element in A:
    if element<m:
        m=element
    if element>M:
        M=element

Let $m$ be the minimum and $M$ be the maximum. Allocate an array $B$ of size $M-m+1$ and initialize it to all 0. If we call $r$ the difference $M-m$, the complexity for this step is $O(r)$.

In [19]:
B=[0]*(M-m+1)
r=M-m

Finally, it is possible to look at every element of the first array $A$ and for each element $e$ found increment the element in position $e-m$ in the second array $B$. This is of complexity $O(n)$.

In [20]:
for element in A:
    B[element-m]+=1

It is now possible to construct an array $C$ which is array $A$ sorted by simply looking at the numbers found in the $B$ array: a number $k$ in position $j$ means that the sorted array will have $k$ copies of the number $j+e$. Complexity for this step is $O(n+r)$

In [21]:
sorted_A=[]

for i in range(len(B)):  #O(r)
    for j in range(B[i]): 
        # The complexity of this cycle is variable with average O(n/r). This is because the number
        # of elements that can get added in the "sorted_A" array is exacly n and this number has to be split into r parts.
        sorted_A.append(i+m)

The array A is now sorted.

In [22]:
print(sorted_A)

[1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 5, 6, 7, 9, 11]


Since the steps are of complexity $O(n), O(r), O(n+r)$ the total complexity of the algorithm is $O(n+r)$.